In [1]:
from IPython.display import Markdown, display
from dotenv import load_dotenv 
from bs4 import BeautifulSoup
import requests
import os
from openai import OpenAI

In [6]:
load_dotenv()

True

In [7]:
api_key = os.getenv('OPENAI_API_KEY')

In [8]:
openai = OpenAI()

In [146]:
class Website:

    def __init__(self, url):
        self.url = url

        content = requests.get(self.url).content

        soup = BeautifulSoup(content, 'html.parser')

        for tag in soup(['script', 'style', 'img', 'input']):
            tag.decompose()

        self.title = soup.title.string

        self.text = soup.body.get_text(separator='\n', strip=True)

In [149]:
website = Website(url)

In [170]:
system_prompt = 'You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown.'

In [171]:
def user_prompt_for(website):

    user_prompt = f"You are looking at a website titled {website.title}. \
The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"

    user_prompt += website.text
    return user_prompt

In [172]:
def messages_for(website):
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt_for(website)},
    ]

In [173]:
def summarize(url):

    website = Website(url)

    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages_for(website)
    )

    return response.choices[0].message.content

In [178]:
summary = summarize('https://www.cnn.com/')

In [177]:
display(Markdown(summary))

# CNN Website Summary

CNN's website provides comprehensive coverage of **breaking news**, latest updates, and videos across various topics, including politics, business, health, entertainment, and sports. 

## Key Topics Covered:
- **2024 Elections**: Insights and updates on the upcoming presidential elections in the U.S.
- **International Affairs**: Coverage on global events, such as the Ukraine-Russia War and the Israel-Hamas War.
- **Politics**: Discussions surrounding significant legislative actions and political figures, including analyses on Trump administration developments and controversial nominees.
- **Health**: Articles on recent health trends, wellness advice, and medical innovations.
- **Weather and Climate**: Updates on weather events like bomb cyclones and their impacts on various regions.

## Recent News Highlights:
- **Ukraine's Recent Actions**: Ukraine has launched U.S.-made longer-range missiles into Russia following permission from the Biden administration.
- **Political Developments**: Trump reportedly intends to nominate Lutnick for the position of Commerce Secretary amidst ongoing ethical inquiries involving House members.
- **Crime Reports**: Coverage on serious crime incidents, including stabbings in Manhattan and shootings in New Orleans.

The website also features a range of multimedia content, including videos, live broadcasts, and podcasts covering current affairs.